# Solving the Poisson-Boltzmann equation

## Finite difference approximation method

#### Used to enforce Dirichlet boundary conditions.

The Poisson-Boltzmann equation can be solved numerically using a finite dfference approximation for the second order partial differential equation. To solve the Poisson-Boltzmann equation numerically, the second order partial derivatives need to be replaced with second-order finite difference approximations using Taylor series expansion. The differential equations then become a system of linear algebraic equations, which can be solved using matrix algebra.

In this work, the sites described are the one-dimensional projection of defects in a crystal lattice onto a grid. In real crystals, lattice sites are not necessarily equally spaced and therefore the finite difference equations must be defined to take into account different $\Delta x$ spacings.

<img src="Figures/irregulargrid.png">  INCLUDE FIGURE OF GRID not working. 

For an irregularly spaced grid as shown above, the first and second derivatives are given as

$$ f'_{x,a} = \frac{f_{x0} - f_{x-1}}{\Delta x_1} = \frac{f_{x,a} + \frac{1}{2} \Delta x_1 - f_{x,a} - \frac{1}{2} \Delta x_1}{\Delta x_1} $$

and

$$ f''_0 = \frac{ f'_{x,b} - f'_{x,a} }{ \frac{1}{2}(\Delta x_1 + \Delta x_2 )}$$

respectively. The second derivative expands to

$$ f''_0 = \frac{ f'_{x,a} - f'_{x,b} }{ \frac{1}{2} ( \Delta x_1 + \Delta x_2 ) } $$

which can be rearranged to give 

$$ f''_0 = \frac{2 ( \Delta x_2 ( f_{x0} - f_{x-1} ) - \Delta x_1(f_{x+1} - f_{x0} ) )}{\Delta x_1 \Delta x_2 (\Delta x_1 + \Delta x_2 ) }$$

Taking the second derivative finite difference at every point in $[x_0, x_2]$ then gives the following expression.

$$ f''_0 = \frac{-2}{\Delta x_1 \Delta x_2 (\Delta x_1 +\Delta x_2 ) } . \Delta x_2f_{x-1} - ( \Delta x_1 + \Delta x_2 ) f_0 + \Delta x_1 f_{x+1} $$

To solve the Poisson-Boltzmann equation, the function at each position is the electrostatic potential and the second derivative is solved to calculate the charge density at each position. Substituting $\Phi$ and $\rho$ yields the equation

$$-\rho = \frac{-2}{\Delta x_1 \Delta x_2 (\Delta x_1 +\Delta x_2 ) } . \Delta x_2\Phi_{x-1} - ( \Delta x_1 + \Delta x_2 ) \Phi_0 + \Delta x_1 \Phi_{x+1} $$

To find the solution of the discretised Poisson-Boltzmann equation, the above expression must be satisfied at all points on the grid. Therefore, these equations become a system of linear algebraic equations solved using matrix inversion. Matrix inversion is known as the process of finding matrix $b$ which satisfies the equation of an invertible matrix $A$. 

Taking prefactor ($p$) as:

$$ p = \frac{-2}{\Delta x_1 \Delta x_2 (\Delta x_1 +\Delta x_2 ) } $$

$A$ as:

$$ A = \Delta x_2\Phi_{x-1} - ( \Delta x_1 + \Delta x_2 ) \Phi_0 + \Delta x_1 \Phi_{x+1} $$

and $b$ as:

$$ b = -\rho $$

The Poisson-Boltzmann equation can be solved. 

The invertible matrix $A$ becomes:

$A$ =
$\begin{pmatrix}
(\Delta x_i + \Delta x_{i+1}) & \Delta x_{i+1} & 0 & 0 & 0 \\
\Delta x_i & (\Delta x_i + \Delta x_{i+1}) & \Delta x_{i+1} & 0 & 0 \\
0 & \Delta x_i & (\Delta x_i + \Delta x_{i+1}) & \Delta x_{i+1} & 0 \\
0 & 0 & \Delta x_i & (\Delta x_i + \Delta x_{i+1}) & \Delta x_{i+1} \\
0 & 0 & 0 & \Delta x_i & (\Delta x_i + \Delta x_{i+1})
\end{pmatrix}$

To account for the prefactor, a new matrix $L$ is constructed. $L$ is created by transposing matrix $A$, multiplying it by the prefactor and then transposing the matrix again.

$$ L = ( A^T . p )^T $$

Vector $b$ is created from values of $\rho$

$b$ =
$\begin{pmatrix}
\rho_1 \\
\rho_2 \\
\rho_3 \\
\rho_4 \\
\rho_5 \\
\end{pmatrix}$

$ \Phi = L^{-1} b $ is solved using ```numpy.linalg.solve``` which gives $\Phi_{new}$. This process is iterated until a preset convergence limit between is met.

To control numerical stability in reaching the convergence limit, a damping factor is used to reduce the jump in values of $\Phi$ between iterations. 

$$ \Phi_{i+1} = \alpha\Phi_{i+1} + ( 1.0 - \alpha)\Phi_i $$

## Cumulative sum method

#### Used to enforce periodic boundary conditions. 

The Poisson-Boltzmann equation can also be solved numerically using a double cumulative sum to solve the second order partial differential equation. 

1. Using an initial guess for the potential at all sites (arbitrarily zero) the concentration at each site is calculated using $ c_{i,x} = c_{i,\infty} \exp \frac{-z_i F \Phi_x + \mu_x }{k_BT} $ and then the charge density at each site is calculated using $ \rho_x = \sum_{i} c_{i,x} z_i F $. 

2. The potential at each site is updated using a double cumulative sum over the charge density at each site scaled by the width of the site (```delta_x```).

 The potential is then defined as the potential calculated using the double cumulative sum plus a constant, 

 $$ \Phi^{\prime} = \Phi + c $$

 and the total charge ($\rho_{total}$) is defined as the sum of the charge density at each site calculated using $\Phi^{\prime}$.

3. ```scipy.optimize.minimize_scalar``` is used to find the value of $c$ which minimises $\rho_{total}$$^2$.

4. The value of $c$ which is calculated is added to $\Phi$ at each site to give $\Phi^{\prime}$ at each site. 

5. Due to the numerical instability of the calculation, a doping factor is also used to update $\Phi$ using this method. 

 $$ \Phi = \alpha\Phi^{\prime} + ( 1.0 - \alpha)\Phi^{\prime} $$
 
6. The updated value of $\Phi$ is used to recalculate the charge density and the process is repeated until the convergence limit is met. 